In [17]:
import os
import sys
import matplotlib.pyplot as plt
import cv2
import numpy as np
import openpyxl

# Ajouter le chemin du dossier contenant les scripts Python
sys.path.append(os.path.abspath('.'))

# Importer les fonctions depuis les scripts Python
from orthogonal_lines_ratio import find_longest_orthogonal_lines, process_images
# Fonction pour visualiser les résultats
def visualize_results(image, box, ratio, image_file):
    plt.figure()
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.plot(box[:, 0], box[:, 1], 'r-', linewidth=2)
    plt.title(f"{image_file} - Ratio: {ratio:.2f}")
    plt.show()

ImportError: cannot import name 'process_images' from 'orthogonal_lines_ratio' (/Users/noejalabert/Documents/GitHub/noe-test-fork/dataetia/data_processing/orthogonal_lines_ratio.py)

In [ ]:
# Définir les chemins
image_folder = '../train/images_1_to_250'
mask_folder = '../train/masks'
output_file = 'ratios_orthogonal.csv'
excel_file = '../train/classif.xlsx'

# Traiter les images et obtenir les résultats
results = process_images(image_folder, mask_folder, output_file)

NameError: name 'process_images' is not defined

In [ ]:
# Prendre le résultat de la première image
if results:
    first_result = results[0]
    image_file, ratio, box, image = first_result
    
    # Visualiser le résultat pour la première image
    visualize_results(image, box, ratio, image_file)

    # Charger le fichier Excel
    workbook = openpyxl.load_workbook(excel_file)
    sheet = workbook.active

    # Ajouter les résultats dans le fichier Excel
    new_row = [image_file, ratio]
    sheet.append(new_row)

    # Sauvegarder les modifications dans le fichier Excel
    workbook.save(excel_file)
    print(f"Results for {image_file} saved to {excel_file}")
else:
    print("No valid results to save.")